In [2]:
from pandas.io.json import json_normalize
import pandas as pd

pd.__version__


u'0.19.0'

In [4]:
df_checkin = pd.read_json('../../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_checkin.json', lines=True)
(df_checkin.head(n=5))

,business_id,checkin_info,type
0,cE27W9VPgO88Qxe4ol6y_g,"{u'9-5': 1, u'7-5': 1, u'13-3': 1, u'17-6': 1,...",checkin
1,mVHrayjG3uZ_RLHkLj-AMg,"{u'11-5': 1, u'9-5': 2, u'9-4': 3, u'15-1': 1,...",checkin
2,mYSpR_SLPgUVymYOvTQd_Q,"{u'5-5': 1, u'5-4': 1, u'9-3': 1, u'11-4': 1, ...",checkin
3,KayYbHCt-RkbGcPdGOThNg,"{u'23-1': 1, u'11-2': 2, u'21-4': 1, u'16-1': ...",checkin
4,b12U9TFESStdy7CsTtcOeg,"{u'5-5': 2, u'13-2': 1, u'8-6': 1, u'5-1': 2, ...",checkin


In [5]:
print len(df_checkin['business_id'].unique())
print len(df_checkin)

61049
61049


This implies all the business ids are unique.

In [7]:
df_business = pd.read_json('../../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json', lines=True)

In [8]:
print len(df_business['business_id'].unique())

85901


Check if any of the 61049 restaurants in checkin dataset are present in the total list of 85901 restaurants. 

All businesses present in the checkin dataset are present in the business dataset. Let's count their checkins.

In [31]:
df_count = pd.DataFrame(index=df_checkin.index, columns=["checkin_count"])
#print df_count.head()
for i,r in df_checkin.iterrows():
    #print df_checkin['checkin_info'][i].values()
    df_count['checkin_count'][i] = sum(df_checkin['checkin_info'][i].values())

print df_count.head(n=5)

  checkin_count
0             9
1            28
2             8
3            65
4             7


In [32]:
print len(df_count)
print "Max num of checkins", df_count.max()
print "Min num of checkins", df_count.min()

61049
Max num of checkins checkin_count    85243.0
dtype: float64
Min num of checkins checkin_count    3.0
dtype: float64


In [33]:
df_new_checkin = pd.concat([df_checkin, df_count], axis=1)

In [34]:
print df_new_checkin.head()

              business_id                                       checkin_info  \
0  cE27W9VPgO88Qxe4ol6y_g  {u'9-5': 1, u'7-5': 1, u'13-3': 1, u'17-6': 1,...   
1  mVHrayjG3uZ_RLHkLj-AMg  {u'11-5': 1, u'9-5': 2, u'9-4': 3, u'15-1': 1,...   
2  mYSpR_SLPgUVymYOvTQd_Q  {u'5-5': 1, u'5-4': 1, u'9-3': 1, u'11-4': 1, ...   
3  KayYbHCt-RkbGcPdGOThNg  {u'23-1': 1, u'11-2': 2, u'21-4': 1, u'16-1': ...   
4  b12U9TFESStdy7CsTtcOeg  {u'5-5': 2, u'13-2': 1, u'8-6': 1, u'5-1': 2, ...   

      type checkin_count  
0  checkin             9  
1  checkin            28  
2  checkin             8  
3  checkin            65  
4  checkin             7  


In [35]:
print df_new_checkin.columns

Index([u'business_id', u'checkin_info', u'type', u'checkin_count'], dtype='object')


Add the checkin count to business dataset.

In [108]:
df_business_chekin = pd.merge(df_business,df_new_checkin, how='left', on=['business_id', 'business_id'])[['attributes','business_id','categories','city','hours','open','review_count', 'stars', 'checkin_count']]

In [109]:
df_business_chekin.head()

,attributes,business_id,categories,city,hours,open,review_count,stars,checkin_count
0,"{u'Take-out': True, u'Drive-Thru': False, u'Ou...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"{u'Tuesday': {u'close': u'21:00', u'open': u'1...",True,7,3.5,NaN
1,"{u'Happy Hour': True, u'Accepts Credit Cards':...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,{},True,5,3.0,NaN
2,{u'Good for Kids': True},cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf, Golf]",Bethel Park,{},False,5,2.5,9
3,"{u'Alcohol': u'full_bar', u'Noise Level': u'av...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"{u'Tuesday': {u'close': u'19:00', u'open': u'1...",True,26,4.5,28
4,"{u'Takes Reservations': True, u'Accepts Credit...",mYSpR_SLPgUVymYOvTQd_Q,"[Active Life, Golf]",Braddock,"{u'Sunday': {u'close': u'15:00', u'open': u'10...",True,3,5.0,8


Fill Nan in checkin_count with 1.

In [110]:
df_business_chekin['checkin_count'].fillna(1, inplace=True)

In [111]:
df_business_chekin.head()

,attributes,business_id,categories,city,hours,open,review_count,stars,checkin_count
0,"{u'Take-out': True, u'Drive-Thru': False, u'Ou...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"{u'Tuesday': {u'close': u'21:00', u'open': u'1...",True,7,3.5,1
1,"{u'Happy Hour': True, u'Accepts Credit Cards':...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,{},True,5,3.0,1
2,{u'Good for Kids': True},cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf, Golf]",Bethel Park,{},False,5,2.5,9
3,"{u'Alcohol': u'full_bar', u'Noise Level': u'av...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"{u'Tuesday': {u'close': u'19:00', u'open': u'1...",True,26,4.5,28
4,"{u'Takes Reservations': True, u'Accepts Credit...",mYSpR_SLPgUVymYOvTQd_Q,"[Active Life, Golf]",Braddock,"{u'Sunday': {u'close': u'15:00', u'open': u'10...",True,3,5.0,8


In [112]:
with open("../out/business_with_checkin_count.json", "w+") as output_file:
    output_file.write(df_business_chekin.to_json())

In [113]:
df = pd.read_json('../out/business_with_checkin_count.json')
df.head()

,attributes,business_id,categories,checkin_count,city,hours,open,review_count,stars
0,"{u'Take-out': True, u'Drive-Thru': False, u'Go...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",1,Dravosburg,"{u'Thursday': {u'close': u'21:00', u'open': u'...",True,7,3.5
1,"{u'Happy Hour': True, u'Accepts Credit Cards':...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],1,Dravosburg,{},True,5,3.0
10,{u'Wi-Fi': u'free'},1_lU0-eSWJCRvNGk78Zh9Q,"[Libraries, Public Services & Government]",5,Carnegie,{},True,4,4.5
100,"{u'Wheelchair Accessible': True, u'Take-out': ...",8Nm_jcCYtMXYW0ODSHDiXA,"[Seafood, Steakhouses, Japanese, Restaurants]",17,Pittsburgh,"{u'Monday': {u'close': u'22:00', u'open': u'16...",True,10,4.0
1000,"{u'Take-out': True, u'Good For': {u'dessert': ...",WYnveDiSEXLCeNkLZrXibQ,"[Bars, Food, Beer, Wine & Spirits, Delis, Nigh...",16,Charlotte,"{u'Monday': {u'close': u'23:00', u'open': u'09...",True,12,3.0
